In [ ]:
import torch

device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

device

### Test Model

In [ ]:
import torch
import pandas as pd 

feature_indexes = [
    1,  # distance_from_last_transaction
    2,  # ratio_to_median_purchase_price
    4,  # used_chip
    5,  # used_pin_number
    6,  # online_order
]

label_indexes = [
    7  # fraud
]

# test_df = pd.read_csv('data/test_sample.csv', )
test_df = pd.read_csv('data/test.csv', )
test_labels_df = test_df.iloc[:, label_indexes]
test_data_df = test_df.iloc[:, feature_indexes]
test_data_df_tensor = torch.tensor(test_data_df.values, dtype=torch.float).to(device)
test_labels_df_tensor = torch.tensor(test_labels_df.values, dtype=torch.float).to(device)

In [ ]:
import torch
from torch import nn

# like scikit learn standard scaler
class TorchStandardScaler:
    def __init__(self):
        self.mean = None
        self.std = None

    def fit(self, tensor):
        self.mean = tensor.mean(dim=0, keepdim=False)
        self.std = tensor.std(dim=0, keepdim=False)

    def transform(self, tensor):
        return (tensor - self.mean) / self.std

    def fit_transform(self, tensor):
        self.fit(tensor)
        return self.transform(tensor)

    
class NeuralNetwork(nn.Module):
    def __init__(self, scaler):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid(),
        )
        self.scaler = scaler

    def forward(self, x):
        with torch.no_grad():
            x_pre = self.scaler.transform(x)
        probs = self.linear_relu_stack(x_pre)
        return probs

    
model = torch.load("model.pth")

# model = torch.load("models/fraud/1/model.pth")
model.eval()
with torch.inference_mode():
    y_pred = model(test_data_df_tensor)


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score
from matplotlib import pyplot as plt

correct = torch.eq(torch.round(y_pred), test_labels_df_tensor).sum().item()
acc = (correct / len(y_pred)) * 100
precision = precision_score(test_labels_df_tensor, torch.round(y_pred))
recall = recall_score(test_labels_df_tensor, torch.round(y_pred))

print(f"Eval Metrics: \n Accuracy: {acc:>0.1f}%, "
      f"Precision: {precision:.4f}, Recall: {recall:.4f} \n")

c_matrix = confusion_matrix(torch.Tensor.cpu(test_labels_df_tensor),
                            torch.round(torch.Tensor.cpu(y_pred)))
ConfusionMatrixDisplay(c_matrix).plot()
